In [ ]:
# Import libraries

# math library
import numpy as np

# visualization library
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt

# machine learning library
from sklearn.linear_model import LinearRegression

# 3d visualization
from mpl_toolkits.mplot3d import axes3d

#import k-fold
from sklearn.model_selection import KFold

from scipy.optimize import minimize

from sklearn.preprocessing import PolynomialFeatures

# remove warning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
data = np.loadtxt('zscore_data.txt', delimiter=';')

# number of training data
n = data.shape[0] #YOUR CODE HERE
print(n)

#select the train data and test data
idx = np.random.permutation(range(n))
idx_test = idx[:271]
idx_train = idx[271:1471]
#print(idx_train)
#print(len(idx_train))
#print(len(idx_test))
train_data = data[idx_train,:]
test_data = data[idx_test, :]

#input the data, X
X = np.ones([1200, 2])
X [:, 0] = train_data[:, 0]
X [:, 1] = train_data[:, 1]
#print(X.shape)

#input y
y = np.ones([1200, 1])
y [:, 0] = train_data[:, 2]
#print (y)

In [ ]:
#define the predictive function
def pred_f (X, w):
    y_pred = X.dot(w)
    return y_pred
#print(pred_f(X,w))

In [ ]:
#define the loss function
def loss_reg(w,X,y,reg): 
    n = len(y)
    w = w.squeeze()[:,None]
    y_pred = pred_f(X,w)
    loss = 1/n* (y_pred - y).T.dot(y_pred - y) 
    d = w.shape[0]
    loss += reg/d * w.T.dot(w)
    return loss
#print(mse_f(X, w, y))

In [16]:
#compute optimum w_solution
def compute_w_solution(X,y,reg):  #the method to determine w by minimizing the loss
    initial_w = np.zeros((X.shape[1],1))
    result = minimize(loss_reg, initial_w, args=(X,y,reg), method='Powell', options={'maxiter':100})
    w_solution = np.array(result.x)[:,None]
    loss_solution = loss_reg(w_solution,X,y,reg)
    return w_solution, loss_solution

In [17]:
#draw graph determine hyper-parameter
def loss_plot (hyper_para, loss_tab):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

   # Generate the values
    x_vals = hyper_para[:, 0]
    y_vals = hyper_para[:, 1]
    z_vals = loss_tab

   # Plot the values
    ax.scatter(x_vals, y_vals, z_vals, c = 'b', marker='o')
    ax.set_xlabel('d, learning capacity')
    ax.set_ylabel('reg, regularization parameter')
    ax.set_zlabel('loss')

In [24]:
#create a mesh grid
list_d = list(range(1,11))
m1, m2 = np.meshgrid(list_d, np.linspace(0,10,100)) # create meshgrid
hyper_para = np.ones([np.prod(m1.shape),2]) 
hyper_para[:,0] = m1.reshape(-1)
hyper_para[:,1] = m2.reshape(-1)
print(hyper_para)
print(hyper_para.shape)

[[  1.   0.]
 [  2.   0.]
 [  3.   0.]
 ..., 
 [  8.  10.]
 [  9.  10.]
 [ 10.  10.]]
(1000, 2)


In [ ]:
#determine the optimum hyper-parameter, by K-fold
kf = KFold(n_splits=4)
train_loss_tab = np.zeros(hyper_para.shape[0])
val_loss_tab = np.zeros(hyper_para.shape[0])
mean_loss_tab = np.zeros(hyper_para.shape[0])
for train_index,test_index in kf.split(X):
        #print("Train Index:",train_index,",Test Index:",test_index)
        train_X, val_X = X[train_index],X[test_index]
        train_y, val_y = y[train_index],y[test_index]
        for i in range(hyper_para.shape[0]):
            d = int(hyper_para[i, 0])
            poly = PolynomialFeatures(d)
            train_X_poly = poly.fit_transform(train_X)
            train_w_solution, train_loss_solution = compute_w_solution(train_X_poly,train_y,hyper_para[i, 1])
            train_loss_tab[i] = train_loss_tab[i] + train_loss_solution
            val_X_poly = poly.fit_transform(val_X)
            val_loss = loss_reg(train_w_solution,val_X_poly,val_y,hyper_para[i, 1])
            val_loss_tab[i] = val_loss_tab[i] + val_loss
            mean_loss_tab[i] = mean_loss_tab[i] + train_loss_tab[i] + val_loss_tab[i]
train_loss_tab = train_loss_tab/4
val_loss_tab = val_loss_tab/4
mean_loss_tab = mean_loss_tab/4
loss_plot (hyper_para, train_loss_tab)
loss_plot (hyper_para, val_loss_tab)
loss_plot (hyper_para, mean_loss_tab)
print(mean_loss_tab)

In [ ]:
#